# ライブラリ

In [2]:
import pandas as pd
import numpy as np 
import datetime
from tqdm.notebook import tqdm 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import time
from urllib.request import urlopen
import copy
import optuna.integration.lightgbm as lgb_o

# 基本データ

In [80]:
results_19 = pd.read_pickle('results_19.pickle')
results_20 = pd.read_pickle('results_20.pickle')
results_21 = pd.read_pickle('results_21.pickle')
results_all = pd.read_pickle('results_all.pickle')

#horse_results = pd.read_pickle('horse_results-Copy1.pickle')

peds = pd.read_pickle('peds.pickle')
peds_20 = pd.read_pickle('peds_20.pickle')
peds_21 = pd.read_pickle('peds_20.pickle')
peds_h = pd.read_pickle('peds_h.pickle') #all_pdes_data

return_tables = pd.read_pickle('Return_tables.pickle')

In [81]:
return_tables

,0,1,2,3
201901010101,単勝,1,140,1
201901010101,複勝,1br3br4,110br110br470,1br2br7
201901010101,枠連,1 - 3,190,1
201901010101,馬連,1 - 3,190,1
201901010101,ワイド,1 - 3br1 - 4br3 - 4,"120br840br1,100",1br12br13
...,...,...,...,...
201910021212,馬連,2 - 14,3530,15
201910021212,ワイド,2 - 14br14 - 15br2 - 15,"1,310br1,650br4,970",17br23br45
201910021212,馬単,14 → 2,6020,27
201910021212,三連複,2 - 14 - 15,20460,69


レースデータ

In [3]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
        レースIDのリスト

        Returns:
        ----------
        race_results_df : pandas.DataFrame
        全レース結果データをまとめてDataFrame型にしたもの
        """
        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(0.6)
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                #メインとなるテーブルデータを取得
                df = pd.read_html(url)[0]

                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")

                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)

                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list

                #インデックスをrace_idにする
                df.index = [race_id] * len(df)

                race_results[race_id] = df
            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df


In [ ]:
#これなら途中再開が可能　to_dataframe関数を行う必要がある
#scrapeでdict dataframeで変換して保存できる

class Results:
    @staticmethod
    def scrape(race_id_list, pre_race_id_list = {}):
        
        #race_id をkeyにしてdataframe型を格納
        race_results = pre_race_id_list
        for race_id in tqdm(race_id_list):
            if race_id in race_results.keys():
                continue
            time.sleep(0.5)
            try:
                time.sleep(0.5)
                url = "https://db.netkeiba.com/race/" + race_id
                #メインになるテーブルデータを取得
                df = pd.read_html(url)[0]
                html = requests.get(url)
                html.encoding = "EUC-JP"
                soup = BeautifulSoup(html.text, "html.parser")

                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div" ,attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df ["race_type"] = [text] * len(df)
                    if "障" in text:
                        df ["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df ["cource_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                    if text in ["良","稍重","重","不良"]:
                        df ["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df ["weather"] = [text] * len(df)
                    if "年" in text:
                        df ["data"] = [text] *len(df)

                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])

                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary":"レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])

                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list

                #インデックスをrace_idにする
                df.index = [race_id] * len(df)

                race_results[race_id] = df

            #存在しないrace_idを飛ばす
            except IndexError:
                continue
            #中断された時にデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #jupyterの停止をした場合
            except:
                break
                
        
        #pd.DataFrame型にして一つのデータにまとめる
        #race_results_df = pd.concat([race_results[key] for key in race_results])
        
        return race_results

    def to_dataframe(race_results):
        race_results_df = pd.concat([race_results[key] for key in race_results])
        
        return race_results_df
                        
    

In [99]:
race_id_list = []
for place in range(1,11,1):
    for kai in range(1,13,1):
        for day in range(1,13,1):
            for r in range(1,13,1):
                race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
                         

馬の成績データ

In [8]:
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """
        
        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3三番目に受賞歴テーブルがくるため、4番目のデータを取得する
                if df.columns[0]=="受賞歴":
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして1つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        
        return horse_results_df


In [10]:
horse_id_list = results_all['horse_id'].unique()
horse_results = HorseResults.scrape(horse_id_list)
horse_results

  0%|          | 0/19775 [00:00<?, ?it/s]

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2017105318,2021/09/11,4中山1,晴,8,3歳以上1勝クラス,NaN,16,4,8,6.7,...,1.9,**,3-3-2-2,36.2-39.6,41.3,546(+14),NaN,NaN,ホウオウセレシオン,NaN
2017105318,2021/07/17,1福島5,晴,9,南相馬特別(1勝クラス),NaN,16,5,10,5.2,...,0.9,**,3-3-3-2,34.9-35.9,36.5,532(-38),NaN,NaN,フィデリオグリーン,NaN
2017105318,2020/12/26,5中山7,晴,9,立志賞(1勝クラス),NaN,18,6,12,3.5,...,1.9,**,1-1-1-1,37.5-36.8,38.7,570(+18),NaN,NaN,ルトロヴァイユ,NaN
2017105318,2020/08/02,1札幌4,晴,7,3歳以上1勝クラス,NaN,8,8,8,1.5,...,0.2,**,7-6-4-4,36.6-34.7,34.5,552(+28),NaN,NaN,シャムロックヒル,110.0
2017105318,2019/08/31,2札幌5,曇,11,札幌2歳S(G3),NaN,12,7,10,1.8,...,0.6,**,10-7-7-2,36.0-37.3,37.3,524(+6),NaN,NaN,ブラックホール,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016104843,2019/07/27,1札幌1,曇,2,3歳未勝利,NaN,8,1,1,19.8,...,0.2,**,8-7,34.9-35.9,34.6,474(+4),NaN,NaN,イエスサンキュー,200.0
2016104843,2019/07/06,3中京3,曇,4,3歳未勝利,NaN,16,5,10,11.2,...,2.9,**,10-9-9,35.4-35.7,37.8,470(+4),NaN,NaN,サドキンザン,NaN
2016104843,2019/02/16,1小倉3,曇,3,3歳未勝利,NaN,18,7,15,12.6,...,2.7,**,8-10-10-10,35.1-36.1,37.9,466(-6),NaN,NaN,ナリス,NaN
2016104843,2019/01/20,1京都7,小雨,5,3歳未勝利,NaN,16,1,2,7.0,...,0.8,**,5-5,35.6-35.7,35.9,472(-2),NaN,NaN,シトラスノート,50.0


In [16]:
horse_results = pd.read_pickle('horse_results.pickle')
horse_results

251337

馬の血統データ

In [34]:
class Peds:
    @staticmethod
    def scrape(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """
        
        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                df = pd.read_html(url)[0]
                
                #重複を削除する処理
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([i], axis=1, inplace=True)
                    df = df.drop_duplicates()
                ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
                
                peds_dict[horse_id] = ped.reset_index(drop=True)
                time.sleep(0.6)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
                
        #列名をpeds_0, ..., peds_61にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')
        
        return peds_df
                

In [48]:
horse_id_list = results_21['horse_id'].unique()
peds_21 = Peds.scrape(horse_id_list)

  0%|          | 0/10097 [00:00<?, ?it/s]

In [49]:
peds_21

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2018105875,ダイワメジャー 2001 栗毛 [血統][産駒] Halo系,カシュカシュ 2009 鹿毛 [血統][産駒] FNo.[1-k],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],スカーレットブーケ 1988 栗毛 [血統][産駒],アフリート Afleet(加) 1984 栗毛 [血統][産駒],スリーソウツ Three Thoughts(米) 1992 鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],ノーザンテースト Northern Taste(加) 1971 栗毛 [血統][産駒],スカーレットインク Scarlet Ink(米) 1971 栗毛 [血統][産駒],...,Green Ticket,Ways to Learn,Turn-to,Nothirdchance,Cosmic Bomb,Almahmoud,Bold Ruler,Somethingroyal,Lyphard,Three Roses
2018105258,エピファネイア 2010 鹿毛 [血統][産駒] Roberto系,ペルレンケッテ 2008 鹿毛 [血統][産駒] FNo.[4-n],シンボリクリスエス 1999 黒鹿毛 [血統][産駒],シーザリオ 2002 青毛 [血統][産駒],ディープインパクト 2002 鹿毛 [血統][産駒],プンティラ Puntilla(独) 1997 黒鹿毛 [血統][産駒],Kris S. 1977 黒鹿毛 [血統][産駒],Tee Kay 1991 黒鹿毛 [血統][産駒],スペシャルウィーク 1995 黒鹿毛 [血統][産駒],キロフプリミエール Kirov Premiere(英) 1990 鹿毛 [血統][産駒],...,Busted,Highclere,Literat,Surama,Aggressor,Raven Locks,イルドブルボン,Liranga,Abary,Pedra
2018101134,ミッキーアイル 2011 鹿毛 [血統][産駒] Halo系,ザッハトルテ 2007 芦毛 [血統][産駒] FNo.[7-a],ディープインパクト 2002 鹿毛 [血統][産駒],スターアイル 2004 鹿毛 [血統][産駒],クロフネ 1998 芦毛 [血統][産駒],パフェ Parfait(米) 1996 鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],ロックオブジブラルタル 1999 鹿毛 [血統][産駒],アイルドフランス Isle de France(米) 1995 鹿毛 [血統][産駒],...,Icecapade,コレラ,Raise a Native,Gold Digger,Nureyev,Pasadoble,Prince John,Prayer Bell,Nashua,Cream Pie
2018102791,Kingman (英) 2011 鹿毛 [血統][産駒] Danzig系,ドゥケシーナ Duchessina(米) 2013 鹿毛 [血統][産駒] FNo.[2-d],Invincible Spirit (愛) 1997 鹿毛 [血統][産駒],Zenda 1999 鹿毛 [血統][産駒],Henrythenavigator 2005 黒鹿毛 [血統][産駒],Dreamlike 1997 鹿毛 [血統][産駒],Green Desert 1983 鹿毛 [血統][産駒],Rafha 1987 鹿毛 [血統][産駒],Zamindar 1994 鹿毛 [血統][産駒],Hope 1991 鹿毛 [血統][産駒],...,Irish River,Luv Luvin',Northern Dancer,South Ocean,Secretariat,Crimson Saint,Exclusive Native,Won't Tell You,Nijinsky,La Dame Du Lac
2018101539,カレンブラックヒル 2009 黒鹿毛 [血統][産駒] Halo系,ヒシシャトル 2005 鹿毛 [血統][産駒] FNo.[7-f],ダイワメジャー 2001 栗毛 [血統][産駒],チャールストンハーバー Charleston Harbor(米) 1998 鹿毛 [血統][産駒],ボストンハーバー Boston Harbor(米) 1994 鹿毛 [血統][産駒],ヒシレイホウ 1995 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],スカーレットブーケ 1988 栗毛 [血統][産駒],Grindstone 1993 黒鹿毛 [血統][産駒],Penny's Valentine 1989 栗毛 [血統][産駒],...,Restless Wind,Dors,Northern Dancer,Pas de Nom,Mr. Prospector,Stick to Beauty,Nearctic,Seximee,ポリック,Brabantia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019105283,ハーツクライ 2001 鹿毛 [血統][産駒] Halo系,ダストアンドダイヤモンズ Dust and Diamonds(米) 2008 鹿毛 [血統]...,サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],アイリッシュダンス 1990 鹿毛 [血統][産駒],Vindication 2000 黒鹿毛 [血統][産駒],Majestically 2002 黒鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],トニービン Tony Bin(愛) 1983 鹿毛 [血統][産駒],ビューパーダンス Buper Dance(米) 1983 黒鹿毛 [血統][産駒],...,Hail to Reason,Mysore,Raise a Native,Gold Digger,Secretariat,Tamerett,Northern Dancer,Goofed,Alleged,Olmec
2019104476,キタサンブラック 2012 鹿毛 [血統][産駒] Halo系,ナターレ 2008 芦毛 [血統][産駒] FNo.[16-a],ブラックタイド 2001 黒鹿毛 [血統][産駒],シュガーハート 2005 鹿毛 [血統][産駒],クロフネ 1998 芦毛 [血統][産駒],ロージーチャーム 2001 鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],サクラバクシンオー 1989 鹿毛 [血統][産駒],オトメゴコロ 1990 栗毛 [血統][産駒],...,Icecapade,コレラ,Halo,Wishing Well,Nijinsky,Key Partner,Northern Dancer,Lady Victoria,Caro,Jedina
2019100665,クロフネ 1998 芦毛 [血統][産駒] Deputy Minister系,ラナンキュラス 2007 青毛 [血統][産駒] FNo.[13-a],フレンチデピュティ French Deputy(米) 1992 栗毛 [血統][産駒],ブルーアヴェニュー Blue Avenue(米) 1990 芦毛 [血統][産駒],スペシャルウィーク 1995 黒鹿毛 [血統][産駒],ファレノプシス 1995 黒鹿毛 [血統][産駒],Deputy Minister 1979 黒鹿毛 [血統][産駒],Mitterand 1981 鹿毛 [血統][産駒],Classic Go Go 1978 黒鹿毛 [血統][産駒],Eliza Blue 1983 芦毛 [血統][産駒],...,セントクレスピン,ミスアシヤガワ,Hail to Reason,Bramalea,Graustark,Golden Trail,Storm Bird,Terlingua,Damascus,Fiji
2019101487,トーセンジョーダン 2006 鹿毛 [血統][産駒] ゼダーン系,クラウンアルテミス 2010 黒鹿毛 [血統][産駒] FNo.[23-b],ジャングルポケット 1998 鹿毛 [血統][産駒],エヴリウィスパー 1997 栗毛 [血統][産駒],リンカーン 2000 鹿毛 

In [44]:
peds_20 = pd.read_pickle('peds_20.pickle')
peds_20

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
2018101626,ゴールドシップ 2009 芦毛 [血統][産駒] Halo系,コスモエルデスト 2011 鹿毛 [血統][産駒] FNo.[14-f],ステイゴールド 1994 黒鹿毛 [血統][産駒],ポイントフラッグ 1998 芦毛 [血統][産駒],ブラックホーク 1994 鹿毛 [血統][産駒],ビューティアイ 2005 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ゴールデンサッシュ 1988 栗毛 [血統][産駒],メジロマックイーン 1987 芦毛 [血統][産駒],パストラリズム 1987 黒鹿毛 [血統][産駒],...,Chieftain,Level Sands,Halo,Wishing Well,Carr de Naskra,Mawgrit,Danzig,Razyana,リアルシャダイ,マックスビューティ
2018105193,ディープインパクト 2002 鹿毛 [血統][産駒] Halo系,ヒストリックスター 2005 鹿毛 [血統][産駒] FNo.[9-f],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],ファルブラヴ Falbrav(愛) 1998 鹿毛 [血統][産駒],ベガ 1990 鹿毛 [血統][産駒],Halo 1969 黒鹿毛 [血統][産駒],Wishing Well 1975 鹿毛 [血統][産駒],Alzao 1980 鹿毛 [血統][産駒],Burghclere 1977 鹿毛 [血統][産駒],...,Lithiot,Nenana Road,Kalamoun,State Pension,Hornbeam,Priddy Fair,Nearctic,Natalma,Tom Fool,Brazen
2018104800,スクリーンヒーロー 2004 栗毛 [血統][産駒] Roberto系,マルバイユ Marbye(愛) 2000 鹿毛 [血統][産駒] FNo.[9-f],グラスワンダー 1995 栗毛 [血統][産駒],ランニングヒロイン 1993 鹿毛 [血統][産駒],Marju 1988 黒鹿毛 [血統][産駒],Hambye 1994 鹿毛 [血統][産駒],Silver Hawk 1979 鹿毛 [血統][産駒],Ameriflora 1989 鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ダイナアクトレス 1983 鹿毛 [血統][産駒],...,Welsh Pageant,Electric Flash,Sir Gaylord,Little Hut,Claude,Ribasha,Thatch,Last Call,Mummy's Pet,Goosie-Gantlet
2018102410,キズナ 2010 青鹿毛 [血統][産駒] Halo系,ヴィアレッタ 2011 鹿毛 [血統][産駒] FNo.[14-c],ディープインパクト 2002 鹿毛 [血統][産駒],キャットクイル Catequil(加) 1990 鹿毛 [血統][産駒],Pivotal 1993 栗毛 [血統][産駒],ヴァレッタ Valletta(英) 2006 鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],Storm Cat (米) 1983 黒鹿毛 [血統][産駒],Pacific Princess 1973 鹿毛 [血統][産駒],...,Bustino,Zerbinetta,Danzig,Foreign Courier,Ahonoora,Balidaress,Arctic Tern,Beaune,Nureyev,Unite
2018100828,リオンディーズ 2013 黒鹿毛 [血統][産駒] Mr. Prospector系,スマートエンデバー 2010 栗毛 [血統][産駒] FNo.[12-c],キングカメハメハ 2001 鹿毛 [血統][産駒],シーザリオ 2002 青毛 [血統][産駒],ファルブラヴ Falbrav(愛) 1998 鹿毛 [血統][産駒],ディスコホール 1989 栗毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],スペシャルウィーク 1995 黒鹿毛 [血統][産駒],キロフプリミエール Kirov Premiere(英) 1990 鹿毛 [血統][産駒],...,Lithiot,Nenana Road,Nearctic,Natalma,Victoria Park,Lady Angela,Herbager,Silver Sari,Crimson Satan,Bright Prospect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018109169,スクワートルスクワート Squirtle Squirt(米) 1998 黒鹿毛 [血統][産...,サキノザザ 2009 栗毛 [血統][産駒] FNo.[20-a],Marquetry 1987 栗毛 [血統][産駒],Lost the Code 1990 鹿毛 [血統][産駒],タイムパラドックス 1998 栗毛 [血統][産駒],サキノローレル 2004 鹿毛 [血統][産駒],Conquistador Cielo 1979 鹿毛 [血統][産駒],Regent's Walk 1981 栗毛 [血統][産駒],Lost Code 1984 黒鹿毛 [血統][産駒],Smarter By the Day 1983 黒鹿毛 [血統][産駒],...,ボールドラツド,Tredam,Blushing Groom,I Will Follow,Saint Cyrien,Bold Lady,ノーザンテースト,アスコツトラップ,エイトラックス,ジユテイフア
2017104830,ジャスタウェイ 2009 鹿毛 [血統][産駒] Halo系,ディオニージア Dionisia(米) 2003 黒鹿毛 [血統][産駒] FNo.[22-c],ハーツクライ 2001 鹿毛 [血統][産駒],シビル 1999 鹿毛 [血統][産駒],Tejano Run 1992 栗毛 [血統][産駒],Essie's Maid 1988 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],アイリッシュダンス 1990 鹿毛 [血統][産駒],Wild Again 1980 黒鹿毛 [血統][産駒],シャロン Charon(米) 1987 栗毛 [血統][産駒],...,Green Dancer,Khazaeen,Tom Rolfe,Wavy Navy,Cyane,Rum Bottle Bay,Intent,My Recipe,Irish Lancer,Esoteric
2017101519,トゥザワールド 2011 鹿毛 [血統][産駒] Mr. Prospector系,ブルーライトクロス 2012 鹿毛 [血統][産駒] FNo.[3-e],キングカメハメハ 2001 鹿毛 [血統][産駒],トゥザヴィクトリー 1996 鹿毛 [血統][産駒],ファスリエフ Fasliyev(米) 1997 鹿毛 [血統][産駒],セイカドミノ 1996 栗毛 [血統][産駒],Kingmambo (米) 1990 鹿毛 [血統][産駒],マンファス Manfath(愛) 1991 黒鹿毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],フェアリードール Fairy Doll(米) 1991 栗毛 [血統][産駒],...,Never Bend,Repercussion,Riverman,Irish Star,Northfields,ツリーオブノレッジ,Hail to Reason,Cosmah,Understanding,Mountain Flower
2018106621,メイショウサムソン 2003 鹿毛 [血統][産駒] Sadler's Wells系,メイショウモーレン 2009 栗毛 [血統][産駒] FNo.[10-a],オペラハウス Opera House(英) 1988 鹿毛 [血統][産駒],マイヴィヴィアン 1997 鹿毛 [血統][産駒],ステイゴールド 1994 黒鹿毛 [血統][産駒],プリティタイディ 1998 黒鹿毛 [血統][産駒],Sadl

In [75]:
class Return:
    @staticmethod
    def scrape(race_id_list):
        """
        払い戻し表データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        return_tables_df : pandas.DataFrame
            全払い戻し表データをまとめてDataFrame型にしたもの
        """
        
        return_tables = {}
        for race_id in tqdm(race_id_list):
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                
                #普通にスクレイピングすると複勝やワイドが区切れないで繋がってしまう。
                #そのため、改行コードを文字列brに変換して後でsplitする。
                f = urlopen(url)
                html = f.read()
                html = html.replace(b'<br />', b'br')
                dfs = pd.read_html(html)
                
                #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
                df = pd.concat([dfs[1], dfs[2]])
                
                df.index = [race_id] * len(df)
                return_tables[race_id] = df
                time.sleep(0.6)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
                
        #pd.DataFrame型にして一つのデータにまとめる
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        return return_tables_df

In [86]:
race_id_list = results_all.index

In [87]:
return_tables_19 = Return.scrape(race_id_list)

  0%|          | 0/131723 [00:00<?, ?it/s]

In [1]:
return_tables = Return.scrape(horse_id_list)

NameError: name 'Return' is not defined

In [67]:
def update_data(old, new):
    """
    Parameters:
    ----------
    old : pandas.DataFrame
        古いデータ
    new : pandas.DataFrame
        新しいデータ
    """
    
    filtered_old = old[~old.index.isin(new.index)]
    return pd.concat([filtered_old, new])